In [23]:
import numpy as np

class LinearRegression:
    def __init__(self, dim):
        self.dim = max(1, dim)
        self.weights = np.zeros((1+dim,1))

    def X_T(self,X):
        num_examples = X.shape[0]
        real_X = np.c_[np.ones(num_examples), X]
        return real_X

    def train(self,X,Y):
        real_X = self.X_T(X)
        pinv_X = np.linalg.pinv(real_X)
        self.weights = np.dot(pinv_X,Y)
        
    def predict(self,X):
        real_X = self.X_T(X)
        h = np.matmul(real_X, self.weights)
        return h
        
class LinearRegressionNonLinearTrans(LinearRegression):
    def __init__(self, dim):
        self.dim = (dim+1)*2 - 1 
        self.weights = np.zeros((self.dim + 1, 1))

    def X_T(self,X):
        num_examples = X.shape[0]
        X_mult = np.prod(X, axis=1)
        real_X = np.c_[np.ones(num_examples), X, X_mult, np.square(X)]
        return real_X

class Circle:
    def __init__(self,parameter):
        self.Circle = parameter

    def X_T(self,X):
        num_examples = X.shape[0]
        real_X = np.c_[np.ones(num_examples), np.square(X)]
        return real_X
        
    def label(self, X):
        real_X = self.X_T(X) 
        real_X = np.multiply(self.Circle, real_X)
        sum_X = np.sum(real_X, axis=1)
        return np.sign(sum_X)
    
class NLTtest():
 
    def __init__(self, numpoints, noise_percentage, parameter):
        self.n = numpoints
        self.points = np.random.uniform(-1.0,1.0,(self.n, 2))
        self.noise = noise_percentage
        self.f = Circle(parameter)
        self.LinearRegression = LinearRegression(2)
        self.NonLinearTrans = LinearRegressionNonLinearTrans(2)
        self.labels = self.f.label(self.points)
        self.noisy_flip()
        
    def noisy_flip(self):
        percentage = self.noise
        n_flip = int(self.n * percentage)
        flip_arr = np.r_[np.ones(self.n - n_flip), np.multiply(-1, np.ones(n_flip))]
        np.random.shuffle(flip_arr)
        self.noisy_labels = np.multiply(flip_arr, self.labels)
        
    def regen_points(self, numpoints, percentage):
        self.n = numpoints
        self.noise = percentage
        self.points = np.random.uniform(-1.0,1.0,(self.n, 2))
        self.labels = self.f.label(self.points)
        self.noisy_flip()

    def LinearRegression_train(self):
        self.LinearRegression.train(self.points, self.noisy_labels)
        
    def e_in(self, xw):
        xw = np.sign(xw)
        diff = np.not_equal(xw, self.noisy_labels)
        e_in = diff.mean()
        return e_in
    
    def LinearRegression_ein(self):
        xw = self.LinearRegression.predict(self.points)
        e_in = self.e_in(xw)
        return e_in

    def NonLinearTrans_ein(self):
        xw = self.NonLinearTrans.predict(self.points)
        e_in = self.e_in(xw)
        return e_in
    
    def NonLinearTrans_train(self):
        self.NonLinearTrans.train(self.points, self.noisy_labels)

    
    
def prob(num_exp):
    numpts = 1000
    noise_percentage = 0.1
    parameter = np.array([-0.6, 1, 1])
    numweights = 100
    LinearRegression_ein = np.array([])
    LinearRegressionNonLinearTrans_w = np.array([])
    LinearRegressionNonLinearTrans_eout = np.array([])
    h_NonLinearTrans = NLTtest(numpts, noise_percentage, parameter)
    for i in range(num_exp):
        h_NonLinearTrans.regen_points(numpts, noise_percentage)
        h_NonLinearTrans.LinearRegression_train()
        h_lrein = h_NonLinearTrans.LinearRegression_ein()
        LinearRegression_ein = np.concatenate((LinearRegression_ein,[h_lrein]))
        h_NonLinearTrans.NonLinearTrans_train()
        h_nltlrw = h_NonLinearTrans.NonLinearTrans.weights
        if i < numweights:
            LinearRegressionNonLinearTrans_w = np.concatenate((LinearRegressionNonLinearTrans_w,h_nltlrw))
            
        h_NonLinearTrans.regen_points(numpts, noise_percentage)
        h_nltlreout = h_NonLinearTrans.NonLinearTrans_ein()
        LinearRegressionNonLinearTrans_eout = np.concatenate((LinearRegressionNonLinearTrans_eout,[h_nltlreout]))
  
    LinearRegressionNonLinearTrans_w = LinearRegressionNonLinearTrans_w.reshape(numweights, 6)
    
    avg_LinearRegression_ein = np.average(LinearRegression_ein)
    avg_NonLinearTrans_w = np.average(LinearRegressionNonLinearTrans_w, axis=0)
    avg_NonLinearTrans_eout = np.average(LinearRegressionNonLinearTrans_eout)
    print("average e_in from linear regression: %f" % avg_LinearRegression_ein)
    print("average weights from NonLinearTrans-linear regression:")
    print(avg_NonLinearTrans_w)
    print("")
    print("average e_out from NonLinearTrans-linear regression: %f" % avg_NonLinearTrans_eout)

In [24]:
prob(1000)

average e_in from linear regression: 0.504728
average weights from NonLinearTrans-linear regression:
[-9.94688188e-01 -8.54500794e-04 -1.03113086e-02 -1.20852553e-03
  1.56029297e+00  1.55432601e+00]

average e_out from NonLinearTrans-linear regression: 0.126064
